# vegMapper

## Getting Started

### Requirements

```shell
conda install -y -c conda-forge earthengine-api
pip install -y geemap pandas requests
```

In [1]:
import ee
import geemap
import math
import random
import requests
import pandas as pd
from json import dumps
from io import BytesIO
from os.path import isfile
#from google.colab import files  # Save Google Colab to Google Drive 

### GEE

The next cell imports 

You might get prompted to leave the page -- don't. If your browser wants to navigate away from the Jupyter notebook environment, reject and open a new tab by right-clicking the link shown after running the cell.

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AY0e-g6QtNWDpVUVH9d7iIH43OBhUXU5oP88yeX_xWB2MfjCZudSHrU5Urg

Successfully saved authorization token.


### Housekeeping

In [3]:
!mkdir -p outputs
!mkdir -p tests
#!rm tests/pts.csv

## Inputs

Required inputs:

* the `startDate` and `endDate` of the period of interest (YYYY-MM-DD)
* an input csv with at least two columns `lat, lon, ...`

**Give the path to your input CSV (optional).** A series of random land coordinates will be used if `CSV` is set to *None* (read from *tests/* or retrieved from the [geonames API](https://api.3geonames.org/) and written to *tests/*).

In [4]:
START = '2017-01-01'
STOP  = '2017-12-31'
CSV   = None

**This cell defines a function to request a random land coordinate from the API.**

Call it with no arguments as a test (if you want to)

In [5]:
def rand_coord(url: str="https://api.3geonames.org/nearest.json?", code: str="US", retry: int=5):
    for i in range(1, retry+1):
        try:
            return requests.get(url, {'randomland': code}).json()['nearest']
        except Exception as e:
            print(f"\rERROR: Bad API response. Retry ({i})")
    raise Exception
    
def rand_sample(lon: float, lat: float, rad: float=0.1):
    alpha = 2 * math.pi * random.random()
    r = rad * math.sqrt(random.random())
    x = r * math.cos(alpha) + lon
    y = r * math.sin(alpha) + lat
    return x, y


test_coord = rand_coord()

print(dumps(test_coord, indent=2))

ERROR: Bad API response. Retry (1)
ERROR: Bad API response. Retry (2)
{
  "inlatt": "44.20444",
  "distance": "0.000",
  "timezone": "America/New_York",
  "elevation": "139",
  "region": "Jefferson County",
  "name": "Home Again Farm",
  "state": "US",
  "latt": "44.20444",
  "longt": "-75.76694",
  "city": "Theresa",
  "prov": "New York",
  "inlongt": "-75.76694",
  "altgeocode": "HOMEAGAIN-IYURV"
}


And we can call *rand_sample* five times to get a sample of coordinates in close proximity to each other, if no CSV is provided above to variable `CSV`.

In [6]:
lon = float(test_coord['longt'])
lat = float(test_coord['latt'])

rand_sample(lon, lat)

(-75.76587182839224, 44.2972890808557)

This cell either reads the input CSV, generates it's own input data using the functions, or reads and CSV of points save previously by the ipynb.

In [7]:
if CSV:
    pts = pd.read_csv(CSV)
elif isfile("tests/pts.csv"):
    pts = pd.read_csv("tests/pts.csv")
else:
    pts = pd.DataFrame([dict(zip(['lon','lat'], rand_sample(lon,lat))) for _ in range(5)])
    pts.to_csv("tests/pts.csv", index=False)

pts.describe()

,lon,lat
count,5.000000,5.000000
mean,-75.726158,44.198908
std,0.034030,0.068755
min,-75.760857,44.113054
25%,-75.743752,44.161049
50%,-75.738505,44.182940
75%,-75.714779,44.261867
max,-75.672897,44.275630


## Workflow

### Region of interest (roi)

Get the minimum bounding extent of the points in the input CSV. Add an arbitrary buffer around the minimum extent and then get a ee.Geometry.Rectangle to represent the ROI.

In [8]:
buf = 0.1

roi = ee.Geometry.Rectangle([pts['lon'].min() - buf, 
                             pts['lat'].min() - buf, 
                             pts['lon'].max() + buf, 
                             pts['lat'].max() + buf])

type(roi)

ee.geometry.Geometry

>Hover/click the wrench icon at the top right to manipulate the *geemap* widget. It has tons of cool features, e.g. import vector and raster datasets from your local file system and/or GEE.

Plot the point geometries together with the ROI polygon.

In [9]:
def get_geom(x):
    return ee.Geometry.Point(x['lon'], x['lat'])

M = geemap.Map(center=[pts['lat'].mean(), pts['lon'].mean()], zoom=11)
M.addLayer(roi, {'color': 'd63000'}, 'Input ROI')
M.addLayer(ee.FeatureCollection(pts.apply(get_geom, axis=1).tolist()), name='Input XY')
M.add_layer_control()
display(M)

Map(center=[44.1989079375035, -75.72615815937681], controls=(WidgetControl(options=['position', 'transparent_b…

## Build the stack!

In [13]:
the_stack = {}

### Sentinel-1

Nice documentation on Sentinel-1 datasets: https://developers.google.com/earth-engine/guides/sentinel1

In [14]:
s1 = (ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT")
      .filterDate(START, STOP)
      .filterBounds(roi)
      .filter(ee.Filter.eq("instrumentMode", "IW"))
      .filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING"))
      .mean()).clip(roi)

s1_bands = s1.bandNames().getInfo()
print(s1_bands)

['VV', 'VH', 'angle']


Calculate radar volume index: `4 * VH / (VH + VV)`


In [15]:
s1_radar_volume_index = s1.expression(
    expression="4 * VH / (VH + VV)", 
    opt_map={'VV': s1.select('VV'), 
             'VH': s1.select('VH')}
).rename('radar_volume_index')

the_stack['s1'] = s1.addBands(s1_radar_volume_index)
print("The stack has", len(the_stack), "layers(s).")

The stack has 1 layers(s).


### ALOS2

https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_SAR

In [16]:
alos2 = (ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')
         .select(ee.List(['HV', 'HH']))
         .filterDate(START, STOP)
         .filterBounds(roi)
         .mean()).clip(roi)

alos2_bands = alos2.bandNames().getInfo()
print(alos2_bands)

['HV', 'HH']


Calculate radar volume index: `4 * HV / (HV + HH)`

In [17]:
alos2_radar_volume_index = alos2.expression(
    expression="4 * HV / (HV + HH)", 
    opt_map={'HV': alos2.select('HV'),
             'HH': alos2.select('HH')}
).rename('radar_volume_index')

the_stack['alos2'] = alos2.addBands(alos2_radar_volume_index)
print("The stack has", len(the_stack), "layers(s).")

The stack has 2 layers(s).


### VCF from MODIS

In [18]:
modisTreeCover = (ee.ImageCollection('MODIS/006/MOD44B')
                  .select('Percent_Tree_Cover')
                  .filterDate(START, STOP)
                  .filterBounds(roi)
                  .first()).clip(roi)

the_stack['modis'] = modisTreeCover
print("The stack has", len(the_stack), "layer(s).")

The stack has 3 layer(s).


### NDVI from Landsat

https://developers.google.com/earth-engine/guides/image_math#colab-python_1

In [19]:
def mask_l8sr(image, cloudShadowBitMask=1<<3, cloudBitMask=1<<5):
    qa = image.select('pixel_qa')  # Select QA band and get conditional mask.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudBitMask).eq(0))
    return image.updateMask(mask)  # Return masked image.

l8sr = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
        .filterDate(START, STOP)
        .filterBounds(roi)
        .map(mask_l8sr)
        .median()).clip(roi)

l8sr_bands = l8sr.bandNames().getInfo()
print(l8sr_bands)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']


In [27]:
help(l8sr.select("B1").clip)

Help on method clip in module ee.image:

clip(clip_geometry) method of ee.image.Image instance
    Clips an image to a Geometry or Feature.
    
    The output bands correspond exactly the input bands, except data not
    covered by the geometry is masked. The output image retains the
    metadata of the input image.
    
    Use clipToCollection to clip an image to a FeatureCollection.
    
    Args:
      clip_geometry: The Geometry or Feature to clip to.
    
    Returns:
      The clipped image.



Calculate NDVI and add it as a band to the existing surface reflectance image:

In [20]:
l8_ndvi = l8sr.normalizedDifference(['B5', 'B4']).rename('ndvi')

the_stack['l8'] = l8sr.addBands(l8_ndvi)
print("The stack has", len(the_stack), "layer(s).")

The stack has 4 layer(s).


## Inspect the imagery

Plot bands ... 4 and 5, and the NDVI ... image:

In [21]:
# def _style():
#     if b=="Percent_Tree_Cover":
#         rng = {'min': 0.0, 'max': 100.0}
#     else:
#         rng = {'min': 0.0, 'max': 1.0}

for src, img in the_stack.items():
    for b in img.bandNames().getInfo():
        M.addLayer(img.select(b), name=b)
M

Map(bottom=95438.0, center=[44.180693875072976, -76.02064401349843], controls=(WidgetControl(options=['positio…

## Run the prediction

Some band math on the stack results. (I'm not sure if these are. Priors are dummy values.)

In [19]:
prior_mean = [  0.06491638, 
              -26.63132179, 
                0.05590800, 
              -29.64091620 ]

prior_mean_int = ee.Number(6.07)

prediction = (ee.Image(prior_mean_int)
              #.add((Fmod_tc_aoi.multiply(prior_mean[0]))
              .add(s1_radar_volume_index.select('radar_volume_index').multiply(prior_mean[1]))
              .add(l8_ndvi.select('ndvi').multiply(prior_mean[2]))
              #.add(smooth.select('constant').multiply(prior_mean[3]))))
              ).clip(roi)

predx = prediction.exp()
pred_final = ee.Image(predx.divide(predx.add(1)))
type(pred_final)

ee.image.Image

Add the prediction to the stack:

In [20]:
the_stack['prediction'] = prediction
print("The stack has", len(the_stack), "layer(s).")

The stack has 5 layer(s).


In [35]:
#!conda env export -f environment.yml
name: geo
channels:
  - conda-forge
  - defaults
dependencies:
  - ee
  - geemap
  - math
  - random
  - requests
  - pandas

name: geo
channels:
  - conda-forge
  - defaults
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=1_gnu
  - aiobotocore=1.2.2=pyhd3eb1b0_0
  - aiohttp=3.7.4=py39h3811e60_0
  - aioitertools=0.7.1=pyhd8ed1ab_0
  - anyio=2.2.0=py39hf3d152e_0
  - argon2-cffi=20.1.0=py39h3811e60_2
  - asciitree=0.3.3=py_2
  - async-timeout=3.0.1=py_1000
  - async_generator=1.10=py_0
  - attrs=20.3.0=pyhd3deb0d_0
  - babel=2.9.0=pyhd3deb0d_0
  - backcall=0.2.0=pyh9f0ad1d_0
  - backports=1.0=py_2
  - backports.functools_lru_cache=1.6.3=pyhd8ed1ab_0
  - bleach=3.3.0=pyh44b312d_0
  - bokeh=2.3.1=py39hf3d152e_0
  - boto3=1.17.49=pyhd8ed1ab_0
  - botocore=1.20.49=pyhd8ed1ab_0
  - brotlipy=0.7.0=py39h3811e60_1001
  - bzip2=1.0.8=h7f98852_4
  - c-ares=1.17.1=h7f98852_1
  - ca-certificates=2020.12.5=ha878542_0
  - cached-property=1.5.2=hd8ed1ab_1
  - cached_property=1.5.2=pyha770c72_1
  - cachetools=4.2.2=pyhd8ed1ab_0
  - cartopy=0.18.0=py39h3b23250_13
  - certifi=2020.12.5=py39hf3d152e_1
  - cf

## Scratch

### Scale

In [21]:
help(the_stack['prediction'].projection().nominalScale)

Help on method Projection.nominalScale in Projection:

Projection.nominalScale(*args, **kwargs) method of ee.Projection instance
    Returns the linear scale in meters of the units of this projection, as
    measured at the point of true scale.
    
    Args:
      proj:



In [22]:
the_stack['prediction'].projection().nominalScale().getInfo()

111319.49079327357

In [23]:
proj = {}
for src, img in the_stack.items():
    p = img.projection()
    proj[src] = p.wkt().getInfo()
    print(src, img.projection().nominalScale().getInfo())

s1 111319.49079327357
alos2 111319.49079327357
modis 231.656358264
l8 111319.49079327357
prediction 111319.49079327357


In [24]:
print(proj['s1'])

GEOGCS["WGS 84", 
  DATUM["World Geodetic System 1984", 
    SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], 
    AUTHORITY["EPSG","6326"]], 
  PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], 
  UNIT["degree", 0.017453292519943295], 
  AXIS["Geodetic longitude", EAST], 
  AXIS["Geodetic latitude", NORTH], 
  AUTHORITY["EPSG","4326"]]


In [28]:
help(img.reduceRegion)

Help on method Image.reduceRegion in Image:

Image.reduceRegion(*args, **kwargs) method of ee.image.Image instance
    Apply a reducer to all the pixels in a specific region. Either the reducer
    must have the same number of inputs as the input image has bands, or it
    must have a single input and will be repeated for each band. Returns a
    dictionary of the reducer's outputs.
    
    Args:
      image: The image to reduce.
      reducer: The reducer to apply.
      geometry: The region over which to reduce data.  Defaults to
          the footprint of the image's first band.
      scale: A nominal scale in meters of the projection to work in.
      crs: The projection to work in. If unspecified, the projection of
          the image's first band is used. If specified in addition to
          scale, rescaled to the specified scale.
      crsTransform: The list of CRS transform values.  This is
          a row-major ordering of the 3x2 transform matrix.
          This option is m

In [31]:
help(the_stack['s1'].resample)

Help on method Image.resample in Image:

Image.resample(*args, **kwargs) method of ee.image.Image instance
    An algorithm that returns an image identical to its argument, but which
    uses bilinear or bicubic interpolation (rather than the default nearest-
    neighbor) to compute pixels in projections other than its native projection
    or other levels of the same image pyramid. This relies on the input image's
    default projection being meaningful, and so cannot be used on composites,
    for example. (Instead, you should resample the images that are used to
    create the composite.)
    
    Args:
      image: The Image to resample.
      mode: The interpolation mode to use.  One of 'bilinear' or
          'bicubic'.)



In [ ]:
the_stack['s1'].resample(mode="bilinear")

## Build the table

```python
print(the_stack['l8'].sample(region=tmp.iloc[0], scale=30).getInfo())
```

In [ ]:
def get_val(img, geom):
    return img.reduceRegion(ee.Reducer.mean(), geom, 30).getInfo()

def get_row(geom, buffer_dist: float=0.1, buffer_proj: str=None):
    row_geom = geom.buffer(distance=buffer_dist, proj=buffer_proj)
    row_data = {'metadata': {}}
    for name, img in the_stack.items():
        img_data = get_val(img, geom)
        #img_data = get_val(img, row_geom)
        for band, val in img_data.items():
            row_data[f"{name}_{band}".lower()] = val
        row_data['metadata'][name] = dumps(img.getInfo())
    return row_data

In [ ]:
# Extract values at each point for each image in the stack into a data frame.
data = pd.DataFrame(pts.apply(get_geom, axis=1).apply(get_row).tolist())

# Join the data frame to the original pts data frame.
out = pts.join(data)

out.info()

In [ ]:
out.iloc[0]

In [ ]:
out.describe()

## Export to Google Drive

We used pandas to generate the table so we may write to Google Drive with the `pd.DataFrame.to_csv` method.

```python
# Omit the column of json metadata from the output table.
out_columns = [c for c in out.columns if c != "metadata"]

# Write the selected columns to a csv:
out[out_columns].to_csv("outputs.csv", index=None)
```